In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

from datetime import datetime
from metloom.pointdata import SnotelPointData

from snobedo.lib.dask_utils import start_cluster, client_ip_and_port
from snobedo.snotel import SnotelLocations

from common import SNOBAL_DIR, DATA_DIR, COARSEN_OPTS, \
    use_hvplot, HV_PLOT_OPTS, BOKEH_FONT, LEGEND_OPTS, LINE_STYLE

In [ ]:
client = start_cluster(10, 32)
client_ip_and_port(client)

In [ ]:
year = 2020
water_year = f'wy{year}'

## SNOTEL 

In [ ]:
snotel_sites = SnotelLocations()
snotel_sites.load_from_json(DATA_DIR / 'snotel_sites_exact.json')

In [ ]:
mcs_snotel_point = SnotelPointData("637:ID:SNTL", "MCS")

mcs_snotel = mcs_snotel_point.get_daily_data(
    datetime(2019, 10, 1), datetime(2024, 7, 31),
    [mcs_snotel_point.ALLOWED_VARIABLES.SNOWDEPTH]
)

mcs_snotel['SNOWDEPTH_M'] = mcs_snotel.SNOWDEPTH * 0.0254

mcs_snotel_mt = mcs_snotel['SNOWDEPTH_M'].reset_index().set_index("datetime").tz_convert('US/Mountain')['SNOWDEPTH_M']

## Model

### SMRF

In [ ]:
mcs_snobal = xr.open_mfdataset(
    (SNOBAL_DIR / f'wy202[0,1,2,3,4]' / 'mcs/*/snow.nc').as_posix(),
    preprocess=lambda ds: ds['thickness'],
    parallel=True,
).sel(
    x=snotel_sites.Mores_Creek_Summit.lon, 
    y=snotel_sites.Mores_Creek_Summit.lat,
    method='nearest',
).compute()

In [ ]:
client.shutdown()

# Matplotlib 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
from matplotlib.offsetbox import AnchoredText

In [ ]:
STATION_LABEL = 'Station'
COLORS = {
        'iSnobal': 'cornflowerblue',
        STATION_LABEL: 'seagreen',    
}

def plot_site(data, ax, site_name):
    for key in data:
        if key == STATION_LABEL:
            continue

        ax.plot(
            data[key].time,
            data[key].data.flatten(),
            label=key, 
            color=COLORS[key], 
            alpha=0.9, lw=1
        )
    ax.plot(
        data[STATION_LABEL].index,
        data[STATION_LABEL].values,
        label=key, 
        color=COLORS[STATION_LABEL], 
        alpha=0.9, lw=1.25
    ) 
    
    ax.set_ylabel(r'Snow Depth (m)')

    ax.tick_params(axis='x', which='minor', tick1On=False, tick2On=False)

    ax.set_xlim([plot_range[0], plot_range[-1]])
    ax.set_yticks(np.arange(0, 4))
    ax.set_ylim(bottom=-0.05)
    ax.set_ylim(top=3)
        
    ax.yaxis.set_minor_locator(mticker.MultipleLocator(0.2))

In [ ]:
periods = 36
plot_range = pd.date_range(start=f'{year - 1}-10-01', periods=periods, freq='MS')
xTicks = mdates.DateFormatter('%b-%Y')

figure_opts = dict(figsize=(6,3), dpi=300,)
fig, axes = plt.subplots(1, 1, sharex=True, **figure_opts)
plt.subplots_adjust(hspace=0.05)

plot_site(
    {
        'iSnobal': mcs_snobal['thickness'],
        STATION_LABEL: mcs_snotel_mt,
    },
    axes,
    'MCS'
)

axes.tick_params(axis='x', which='major', pad=0, labelrotation=45)

# at = AnchoredText(
#     f'Water Year {year} - 2022', 
#     prop=dict(size=10), 
#     frameon=False, 
#     loc='upper left', 
#     pad=0.3, 
#     borderpad=0.25,
# )
# axes.add_artist(at)
axes.legend(
    frameon=False,
    bbox_to_anchor=(0.995, 0.99),
    ncol=5,
    borderaxespad=0.15, 
    fontsize=8
);

# HV plot

## SNOTEL 

In [ ]:
import hvplot.xarray
import holoviews as hv

use_hvplot()
hvplot.extension('bokeh')

In [ ]:
hv.output(fig='auto', dpi=300)

HV_PLOT_OPTS['ylabel'] = 'Snow Depth (m)'
HV_PLOT_OPTS['xlabel'] = f'Water Year {year}'

y_lim = (-0.05, 3.)
SNOTEL_LINE=dict(line_width=3)
date_range = np.datetime64(f'{year - 1}-10-20'), np.datetime64(f'2024-06-30')

In [ ]:
mcs_snobal.thickness.hvplot(label='iSnobal').opts(
    title='Mores Creek Summit', ylim=y_lim, xlim=date_range, **LINE_STYLE, **HV_PLOT_OPTS
) * \
mcs_snotel_mt.plot(label='Station', **SNOTEL_LINE)